In [ ]:
import torch
import os, sys
parent_dir = os.path.abspath('../')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from data.data_loader import CreateDataLoader
from models.mdct import IMDCT4
from models.models import create_model
from util.visualizer import Visualizer
import numpy as np
import matplotlib.pyplot as plt
os.environ['CUDA_VISIBLE_DEVICES']='3'

In [ ]:
opt = torch.load('/root/pix2pixHD/generated/opt.obj')
opt.load_pretrain='../checkpoints/hifitts_G5L3_48ngf_arcsinh_fitres2'
opt.gpu_ids = [0]
visualizer = Visualizer(opt)
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
dataset_size = len(data_loader)
model = create_model(opt)
print('#audio segments = %d' % dataset_size)

In [ ]:
from util.util import kbdwin
#from dct.dct import IDCT
#_idct = IDCT()
_imdct = IMDCT4(window=kbdwin, win_length=opt.win_length, hop_length=opt.hop_length, n_fft=opt.n_fft, center=opt.center, out_length=opt.segment_length, device = 'cuda')

E = []
D = []
d = []
lr = []
N = []
model.eval()
with torch.no_grad():
    for i, data in enumerate(dataset):
        sr_spectro, lr_pha, norm_param, lr_spectro = model.inference(data['label'], None)
        lr.append(lr_spectro.cpu().numpy())
        N.append(norm_param)
        denorm_lr_spectro = model._mdct(data['label'])
        D.append(denorm_lr_spectro.cpu().numpy())
        denorm_lr_spectro_ = model.denormalize(lr_spectro, norm_param)
        d.append(denorm_lr_spectro_.cpu().numpy())
        err = (denorm_lr_spectro - denorm_lr_spectro_.squeeze().permute(0,2,1))**2
        err = err.cpu().numpy()
        E.append(err)

In [ ]:
plt.plot(E[0][0,:,:])
plt.show()

In [ ]:
sp_fig, sp_ax = plt.subplots()
DD = D[1][0,:,:]
sp_ax.pcolormesh(np.log10(np.abs(DD[:,:42])), cmap='PuBu_r')
plt.show()

In [ ]:
sp_hist_fig, sp_hist_ax = plt.subplots()
sp_hist_ax.hist(DD.reshape(-1,1),bins=100)
plt.show()

In [ ]:
DD = np.arcsinh(DD[:,:42]*500)/np.log(10)
MM = np.max(DD)
mm = np.min(DD)
sp_hist_fig, sp_hist_ax = plt.subplots()
sp_hist_ax.hist(DD.reshape(-1,1),bins=100)
plt.show()

In [ ]:
print(MM, mm)
print(N[1]['max'],N[1]['min'])

In [ ]:
sp_fig, sp_ax = plt.subplots()
ll=np.transpose(np.squeeze(lr[1])[0,:42,:])
sp_ax.pcolormesh(np.abs(ll), cmap='PuBu_r')
plt.show()

In [ ]:
sp_hist_fig, sp_hist_ax = plt.subplots()
sp_hist_ax.hist(ll.reshape(-1,1),bins=100)
plt.show()

In [ ]:
ll = (ll + 1)/2
ll = ll*(MM-mm) + mm
sp_hist_fig, sp_hist_ax = plt.subplots()
sp_hist_ax.hist(ll.reshape(-1,1),bins=100)
plt.show()

In [ ]:
sp_fig, sp_ax = plt.subplots()
ll = np.sinh(ll*np.log(10))/500
sp_ax.pcolormesh(np.log10(np.abs(ll)), cmap='PuBu_r')
plt.show()

In [ ]:
LL = torch.Tensor(lr[1]).cuda()
LL = (LL-opt.norm_range[0])/(opt.norm_range[1]-opt.norm_range[0])
LL = LL*(N[1]['max']-N[1]['min'])+N[1]['min']
LL = torch.sinh(LL*torch.log(torch.Tensor([10]).cuda()))/opt.arcsinh_gain
LL = LL.squeeze()
sp_fig, sp_ax = plt.subplots()
sp_ax.pcolormesh(np.log10(np.abs(LL[0,:42,:].cpu().numpy())), cmap='PuBu_r')
plt.show()

In [ ]:
LL = torch.Tensor(lr[1]).cuda()
LL = model.denormalize(LL, N[1])
LL = LL.squeeze()
sp_fig, sp_ax = plt.subplots()
sp_ax.pcolormesh(np.log10(np.abs(LL[0,:42,:].cpu().numpy())), cmap='PuBu_r')
plt.show()

In [ ]:
sp_hist_fig, sp_hist_ax = plt.subplots()
sp_hist_ax.hist(LL[0,:42,:].cpu().numpy().reshape(-1,1),bins=100)
plt.show()

In [ ]:
sp_hist_fig, sp_hist_ax = plt.subplots()
sp_hist_ax.hist(ll.reshape(-1,1),bins=100)
plt.show()

In [ ]:
sp_fig, sp_ax = plt.subplots()
err = (ll-DD[:,:42])
sp_ax.pcolormesh(np.log10(np.abs(err)), cmap='PuBu_r')
plt.show()
print(np.mean(err))

In [ ]:
sp_fig, sp_ax = plt.subplots()
dd = np.log10(np.abs(np.transpose(np.squeeze(d[1])[0,:42,:])))
sp_ax.pcolormesh(np.log10(np.abs(np.transpose(np.squeeze(d[1])[0,:42,:]))), cmap='PuBu_r')
plt.show()

In [ ]:
sp_hist_fig, sp_hist_ax = plt.subplots()
sp_hist_ax.hist(dd.reshape(-1,1),bins=100)
plt.show()